# 2021 01 10 zio test start

This is a text cell. Start editing!

In [1]:
import zio.{test => _, _}
import zio.test._
import Assertion._
import TestAspect._
import environment._
import zio.duration._

In [2]:
object mySpec extends DefaultRunnableSpec {
    override def runner: TestRunner[TestEnvironment, Any] =
      defaultTestRunner

    override def spec =
      suite("AdditionSpec")(
        test("addition works") {
          println("some message...")
          assert(1 + 1)(equalTo(32))
        }
      )
  }

mySpec.runner.unsafeRunSync(mySpec.spec)

some message...
- AdditionSpec
  - addition works
    2 did not satisfy equalTo(32)
Ran 1 test in 790 ms: 0 succeeded, 0 ignored, 1 failed


Success(ExecutedSpec(SuiteCase(AdditionSpec,Vector(ExecutedSpec(TestCase(addition works,Left(Assertion(Value(FailureDetails(List(zio.test.AssertionValue$$anon$1@245ec8f9),None)))),Map()))))))

In [3]:
object IntroSpec extends DefaultRunnableSpec {

  val eff2 = for {
    p <- Promise.make[Nothing, Int]
    // _ <- TestClock.adjust(1 second)
    //_     <- ZIO.sleep(1 second)
    _ <- p.succeed(2).fork
    value <- p.await
  } yield value

  override def spec =
    suite("IntroSpec")(
      test("addition works") {
        assert(1 + 1)(equalTo(2))
      } @@ ignore,
      testM("effect test") {
        assertM(eff2)(equalTo(32))
      },
      testM("effect map test") {
        ZIO.succeed(2).map(n => assert(n)(equalTo(2)))
      },
      testM("combination test") {
        for {
          a <- ZIO.succeed(2)
          b <- ZIO.succeed(3)
        } yield assert(a + b)(equalTo(5))
      },
      testM("each result test") {
        for {
          a <- ZIO.succeed(2)
          b <- ZIO.succeed(3)
        } yield assert(a)(equalTo(2)) && assert(b)(equalTo(3))
      },
      testM("ignore this test") {
        ???
      } @@ ignore,
      suite("db tests")(
        test("db1") {
          ???
        } @@ ignore
      ) @@ sequential
    ) @@ parallel // all tests in a suite run in parallel by default
}

In [4]:
IntroSpec.runner.unsafeRunSync(IntroSpec.spec)

Success(ExecutedSpec(SuiteCase(IntroSpec,Vector(ExecutedSpec(TestCase(addition works,Right(Ignored),Map(zio.test.TestAnnotation@a65366a5 -> 1))), ExecutedSpec(TestCase(effect test,Left(Assertion(Value(FailureDetails(List(zio.test.AssertionValue$$anon$1@6

In [5]:
IntroSpec.run

zio.ZIO$FlatMap@5dc487